In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

# Data
levels = np.array([0, 1, 2, 3, 4, 5, 6, 7])
barriers = np.array([17.68, 14.62, 14.84, 13.99, 15.42, 14.35, 14.37, 12.98])
mutations = [
    "49Cys",
    "49Cys + 54",
    "49Sec + 54",
    "49Sec + 54 + 24",
    "49Sec + 54 + 24 + 139",
    "49Sec + 54 + 24 + 139 + 47",
    "49Sec + 54 + 24 + 139 + 47 + 60",
    "49Sec + 54 + 24 + 139 + 47 + 60 + 74"
]

# Surface mock data
y = np.linspace(0, 1, len(levels))
X, Y = np.meshgrid(levels, y)
Z = np.tile(barriers, (len(y), 1))
xi = np.linspace(0, 7, 200)
yi = np.linspace(0, 1, 200)
xi_grid, yi_grid = np.meshgrid(xi, yi)
zi_grid = griddata((X.flatten(), Y.flatten()), Z.flatten(), (xi_grid, yi_grid), method='cubic')

# Colors
colors = cm.get_cmap("Set2")(np.linspace(0, 1, len(levels)))

# Create combined plot
fig = plt.figure(figsize=(25, 10))

# --- 3D Surface with Discrete Path Sampling ---
ax1 = fig.add_subplot(131, projection='3d')
surf = ax1.plot_surface(xi_grid, yi_grid, zi_grid, cmap='YlOrBr', alpha=0.6, edgecolor='none')
for i in range(len(levels)):
    ax1.scatter(levels[i], 0.5, barriers[i], color=colors[i], s=80, label=f"Level {i}: {mutations[i]}")
    if i > 0:
        ax1.plot([levels[i - 1], levels[i]], [0.5, 0.5], [barriers[i - 1], barriers[i]], color=colors[i], linewidth=2.5)
ax1.set_xlabel("Greedy Step", fontsize=12)
ax1.set_ylabel("Landscape Depth (mock)", fontsize=12)
ax1.set_zlabel("Free Energy (kcal/mol)", fontsize=12)
ax1.set_title("3D Energy Landscape with Discrete Path Sampling", fontsize=14)
ax1.view_init(elev=30, azim=-60)
ax1.legend(loc='upper left', bbox_to_anchor=(1.1, 1), fontsize=10, title="Mutant Path", title_fontsize=11)

# --- Bar Plot ---
ax2 = fig.add_subplot(132)
ax2.bar(levels, barriers, color=colors, tick_label=[f"L{i}" for i in levels])
ax2.set_title("Free Energy Barrier per Greedy Step", fontsize=14)
ax2.set_xlabel("Greedy Step", fontsize=12)
ax2.set_ylabel("Free Energy Barrier (kcal/mol)", fontsize=12)
ax2.grid(True, linestyle='--', alpha=0.6)
ax2.set_xticks(levels)
ax2.set_xticklabels([f"Level {i}" for i in levels], rotation=45)

# --- Folding Funnel Plot ---
ax3 = fig.add_subplot(133, projection='3d')
theta = np.linspace(0, 2 * np.pi, 100)
for i in range(len(levels)):
    r = levels[-1] - levels[i] + 1  # radius shrinks toward the bottom of the funnel
    z = barriers[i]
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    ax3.plot(x, y, z*np.ones_like(theta), color=colors[i], label=f"Level {i}")
    ax3.scatter(0, 0, z, color=colors[i], s=60)  # central point at each level

ax3.set_title("Folding Funnel Representation", fontsize=14)
ax3.set_xlabel("X (arbitrary units)")
ax3.set_ylabel("Y (arbitrary units)")
ax3.set_zlabel("Free Energy (kcal/mol)")
ax3.view_init(elev=30, azim=45)
ax3.legend(loc='upper left', fontsize=9)

plt.tight_layout()
plt.show()


/tmp/ipykernel_59120/2356660536.py:31: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = cm.get_cmap("Set2")(np.linspace(0, 1, len(levels)))
